## DS 5559: Group 2 Reddit Scraping

### Sarah Adams, sa4fa
### Claire Setser, cas3ph
### Landon Blakey, ldb3mj

#### Last Modified: October 11, 2020

In [1]:
import pandas as pd
import praw
import psaw
import datetime as dt
import math
import time 
from psaw import PushshiftAPI

In [2]:
r = praw.Reddit(client_id='uYB49_0KLeLg-Q',
                client_secret='', 
                user_agent='DS5559-project', 
                username='OK_Tea_9189', 
                password='')

In [3]:
print(r.user.me())

Ok_Tea_9189


In [4]:
#health subreddits
health_subreddits = ['EDAnonymous', 'addiction', 'alcoholism', 'adhd', 'anxiety', 'autism', 'BipolarReddit', 'bpd', 
        'depression', 'healthanxiety', 'lonely', 'ptsd', 'schizophrenia', 'socialanxiety', 'SuicideWatch', 
        'mentalhealth', 'COVID19_support', 'meditation', 'fitness']

#non-mental health subreddits
non_health_subreddits = ['conspiracy', 'divorce', 'guns', 'jokes', 'legaladvice', 
                         'parenting', 'presonalfinance', 'relationships', 'teaching']


In [5]:
def get_date(created):
    return dt.date.fromtimestamp(created)

In [6]:
def get_submission_from_timestamp(epoch_time, sub):
    start_time = time.gmtime(epoch_time)
    df_dict = {}
    day_list = []
    date = get_date(epoch_time)
    stop = dt.datetime.today().timestamp()
    delta_epoch=int(stop - start)
    
    submissions = list(api.search_submissions(before=epoch_time,
                        subreddit=sub,
                        filter=['url','author', 'title', 'subreddit','created'],
                        limit=30))

    for submission in submissions:
        day_list.append([submission.url, submission.author, submission.title, submission.subreddit, submission.created])
        #print(submission.title)
    df = pd.DataFrame(day_list, columns=['url', 'author', 'title', 'subreddit', 'created'])
    timestamp = df['created'].apply(get_date)
    df['Date'] = timestamp
    df['Date'] =pd.to_datetime(df.Date)
    return df
            

In [7]:
api = PushshiftAPI(r)
start_epoch=int(dt.datetime(2019, 1, 1).timestamp())

submissions = list(api.search_submissions(after=start_epoch,
                            subreddit=['addiction', 'alcoholism'],
                            filter=['url','author', 'title', 'subreddit','created'],
                            limit=100))

In [8]:
subs = ['EDAnonymous', 'addiction', 'alcoholism', 'adhd', 'anxiety', 'autism', 'BipolarReddit', 'bpd', 
        'depression', 'healthanxiety', 'lonely', 'ptsd', 'schizophrenia', 'socialanxiety', 'SuicideWatch', 
        'mentalhealth', 'COVID19_support', 'meditation', 'fitness', 'conspiracy', 'divorce', 'guns', 
        'jokes', 'legaladvice', 'parenting', 'presonalfinance', 'relationships', 'teaching']
days = 273
start = 1577836800
hrs24 = 86400
data = {}
#current_epoch = start + hrs24
#date = get_date(current_epoch)
#fullData = get_submission_from_timestamp(current_epoch, 'addiction')
#fullData.drop(fullData[fullData['Date'] != str(date)].index, inplace = True) 
fullData = pd.DataFrame(data=({'url': ['NA'], 'author': ['NA'], 'title': ['NA'],  'subreddit':['NA'] , 'created':['NA'] , 'Date':['NA']}))
for sub in subs:
    
    for day in range(1,days):
        current_epoch = start + hrs24*day
        date = get_date(current_epoch)
        temp = get_submission_from_timestamp(current_epoch, sub)
        temp.drop(temp[temp['Date'] != str(date)].index, inplace = True)
        fullData = pd.concat([fullData,temp],ignore_index=False)
    fullData.set_index(['Date', 'subreddit','created'])


/opt/anaconda3/envs/python-default/lib/python3.7/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/opt/anaconda3/envs/python-default/lib/python3.7/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [9]:
fullData.to_csv('fullData_SA.csv', index = False)